In [ ]:
from pathlib import Path
import datasets
#dataset_config = 'mimic-cxr','mimic-iii'  
#split = 'train','validate',test

max_input_length = 768

def build_dataset(dataset_config, tokenizer, split):
    data_path = '/nfs/turbo/umms-vgvinodv/data/bioNLP23-Task-1B/data/'
    findings_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.findings.tok')
    impression_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.impression.tok')
    image_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.image.tok')


    findings = [line.strip() for line in open(findings_file_path).readlines()]
    impression = [line.strip() for line in open(impression_file_path).readlines()]
    
    def generate_image_path(line):
        return str(Path(data_path).joinpath(dataset_config).joinpath(line.strip().split(',')[0]))
    image_paths = [generate_image_path(line) for line in open(image_file_path).readlines()]

    dataset = datasets.Dataset.from_dict({"text":findings,"summary":impression}) 
    #dataset = datasets.Dataset.from_dict({"text":findings,"summary":impression,"img_path":image_paths}) 
    
    
    def preprocess_function(samples):
        texts = samples["text"]
        summaries = samples["summary"]
        prompt = "The main impression based on the given FINDINGS section of the chest X-ray report are:"
        
        def generate_input(_text,_summary):
            return " ".join([_text,prompt,_summary])

        inputs = [generate_input(_text,_summary) for _text,_summary in zip(texts,summaries)]
        model_inputs = tokenizer(inputs)
        
        return model_inputs
    
    dataset = dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=list(dataset.features))

    return dataset

In [ ]:
from pathlib import Path
import datasets

dataset_config, split = 'mimic-cxr', 'train'
data_path = '/nfs/turbo/umms-vgvinodv/data/bioNLP23-Task-1B/data/'

findings_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.findings.tok')
impression_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.impression.tok')
image_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.image.tok')


findings = [line.strip() for line in open(findings_file_path).readlines()]
impression = [line.strip() for line in open(impression_file_path).readlines()]

def generate_image_path(line):
    return str(Path(data_path).joinpath(dataset_config).joinpath(line.strip().split(',')[0]))
image_paths = [generate_image_path(line) for line in open(image_file_path).readlines()]

dataset = datasets.Dataset.from_dict({"text":findings,"summary":impression,"img_path":image_paths}) 

In [ ]:
print(dataset["img_path"][0])
print(len(dataset))

def check_img_exists(example):
    return example["img_path"].split('/')[10] != 'p10'

dataset = dataset.filter(check_img_exists)

print(dataset["img_path"][0])
print(len(dataset))

In [ ]:
import skimage, torch, torchvision

# Prepare the image:
img_pth = dataset["img_path"][10]
img = skimage.io.imread(img_pth)
skimage.io.imshow(img) 
skimage.io.show()

In [ ]:
!pip install git+https://github.com/RyanWangZf/MedCLIP.git

In [ ]:
!pip uninstall transformers -y
!pip install transformers

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, MedCLIPVisionModel
from medclip import MedCLIPProcessor
from PIL import Image

sample = dataset[10]
# prepare for the demo image and texts
processor = MedCLIPProcessor()
image = Image.open(sample["img_path"])
text = sample["summary"]
inputs = processor(
    text=text,
    images=image, 
    return_tensors="pt", 
    padding=True
    )

# pass to MedCLIP model
model = MedCLIPModel(vision_cls=MedCLIPVisionModel)
model.from_pretrained()
model.cuda()
outputs = model(**inputs)
print(outputs.keys())
print(outputs['logits'])
# dict_keys(['img_embeds', 'text_embeds', 'logits', 'loss_value', 'logits_per_text'])